# Improved User Model

In [1]:
# Don't!
# from user.models import User
# Do instead:
from django.contrib.auth import get_user_model
User = get_user_model()

In [2]:
# works just like a model
new_user = User(short_name="Andrew", email="django@jambonsw.com")

In [3]:
new_user

<User: django@jambonsw.com>

In [4]:
# with a bit of an exception
new_user.password

''

In [5]:
new_user.set_password('securepassword!')

In [6]:
new_user.password

'argon2$argon2i$v=19$m=512,t=2,p=2$MGdYVkRXZXdIaU5n$PX7tcVmcGLvx6Mw4bNCVPw'

In [7]:
# Don't:
# user = User.objects.create(...)
# user.set_password(...)
# user.save()
# Do:
user = User.objects.create_user(
    email="new_user@jambonsw.com",
    short_name="User",
    full_name="User McUser",
    password="securepassword!",
)

In [8]:
user

<User: new_user@jambonsw.com>

In [9]:
user.password

'argon2$argon2i$v=19$m=512,t=2,p=2$YkhQNHdicmxiQldK$UA1n1rvp5Yz82aOaWY2OBw'

In [10]:
user.check_password("securepassword!")

True

In [11]:
user.check_password("wrong")

False

In [12]:
# Django Girls is a real and great organization
# The email shown here is fake, however!
superuser = User.objects.create_superuser(
    email="ada@djangogirls.org",
    short_name="Ada",
    full_name="Ada Lovelace",
    password="securepassword!",
)

# Content-Type

In [13]:
from django.contrib.contenttypes.models import ContentType

In [14]:
list(ContentType.objects.all())

[<ContentType: log entry>,
 <ContentType: permission>,
 <ContentType: group>,
 <ContentType: content type>,
 <ContentType: session>,
 <ContentType: blog post>,
 <ContentType: tag>,
 <ContentType: startup>,
 <ContentType: news article>,
 <ContentType: user>]

In [15]:
list(ContentType.objects.values())

[{'id': 1, 'app_label': 'admin', 'model': 'logentry'},
 {'id': 2, 'app_label': 'auth', 'model': 'permission'},
 {'id': 3, 'app_label': 'auth', 'model': 'group'},
 {'id': 4, 'app_label': 'contenttypes', 'model': 'contenttype'},
 {'id': 5, 'app_label': 'sessions', 'model': 'session'},
 {'id': 6, 'app_label': 'blog', 'model': 'post'},
 {'id': 7, 'app_label': 'organizer', 'model': 'tag'},
 {'id': 8, 'app_label': 'organizer', 'model': 'startup'},
 {'id': 9, 'app_label': 'organizer', 'model': 'newslink'},
 {'id': 10, 'app_label': 'user', 'model': 'user'}]

In [16]:
Post_CT = ContentType.objects.get(app_label="blog", model="post")

In [17]:
Post_CT

<ContentType: blog post>

In [18]:
Post = Post_CT.model_class()

In [19]:
Post

blog.models.Post

For more info: https://docs.djangoproject.com/en/2.2/ref/contrib/contenttypes/

# Permissions

Format:

- `<app_label>.<behavior>_<model>`
- `<app_label>.<code_name>`

Behavior: `view`, `add`, `change`, `delete`

In [20]:
user.has_perm("blog.add_post")

False

In [21]:
user.has_perm("organizer.change_tag")

False

In [22]:
superuser.has_perm("blog.add_post")

True

In [23]:
superuser.has_perm("organizer.change_tag")

True

In [24]:
user.user_permissions.all()

<QuerySet []>

In [25]:
superuser.user_permissions.all()

<QuerySet []>

In [26]:
user.is_superuser

False

In [27]:
superuser.is_superuser

True

In [28]:
from django.contrib.auth.models import Permission
Permission.objects.count()

40

In [29]:
blog_content_type = ContentType.objects.get_for_model(Post)
list(
    Permission.objects.filter(content_type=blog_content_type)
)

[<Permission: blog | blog post | Can add blog post>,
 <Permission: blog | blog post | Can change blog post>,
 <Permission: blog | blog post | Can delete blog post>,
 <Permission: blog | blog post | Can view blog post>]

In [30]:
list(
    Permission.objects.filter(content_type=blog_content_type).values()
)

[{'id': 21,
  'name': 'Can add blog post',
  'content_type_id': 6,
  'codename': 'add_post'},
 {'id': 22,
  'name': 'Can change blog post',
  'content_type_id': 6,
  'codename': 'change_post'},
 {'id': 23,
  'name': 'Can delete blog post',
  'content_type_id': 6,
  'codename': 'delete_post'},
 {'id': 24,
  'name': 'Can view blog post',
  'content_type_id': 6,
  'codename': 'view_post'}]

In [31]:
new_permission = Permission.objects.create(
    codename='view_future_post',
    name='Can view unpublished Post',
    content_type=blog_content_type,
)

*Warning*: Above is not useful for migrations, or across deployments!

Instead: 

```python
class Post(models.Model):

    class Meta:
        permissions = (
            (
                "view_future_post",
                "Can view unpublished Post",
            ),
        )
```

Follow this code addition with a migration:

`./manage.py makemigrations --name=add_view_future_post_permission blog`

In [32]:
new_permission.delete()

(1,
 {'auth.Group_permissions': 0,
  'user.User_user_permissions': 0,
  'auth.Permission': 1})

In [33]:
user.user_permissions.add(*Permission.objects.filter(content_type=blog_content_type))

In [34]:
user.has_perm("blog.add_post")

False

In [35]:
user.refresh_from_db()

In [36]:
user.has_perm("blog.add_post")

False

In [37]:
user.user_permissions.all()

<QuerySet [<Permission: blog | blog post | Can add blog post>, <Permission: blog | blog post | Can change blog post>, <Permission: blog | blog post | Can delete blog post>, <Permission: blog | blog post | Can view blog post>]>

In [38]:
user.has_perm("blog.add_post")

False

In [39]:
user = User.objects.get(pk=user.pk)

In [40]:
user.has_perm("blog.add_post")

True

# Groups

In [41]:
from django.contrib.auth.models import Group
Group.objects.all()

<QuerySet []>

In [42]:
contributors = Group.objects.create(name='contributors')
Group.objects.values()

<QuerySet [{'id': 2, 'name': 'contributors'}]>

In [43]:
list(contributors.permissions.all())

[]

In [44]:
Tag_CT = ContentType.objects.get(app_label="organizer", model="tag")


In [45]:
list(
    Permission.objects.filter(content_type=Tag_CT).values()
)

[{'id': 25,
  'name': 'Can add tag',
  'content_type_id': 7,
  'codename': 'add_tag'},
 {'id': 26,
  'name': 'Can change tag',
  'content_type_id': 7,
  'codename': 'change_tag'},
 {'id': 27,
  'name': 'Can delete tag',
  'content_type_id': 7,
  'codename': 'delete_tag'},
 {'id': 28,
  'name': 'Can view tag',
  'content_type_id': 7,
  'codename': 'view_tag'}]

In [46]:
contributors.permissions.add(*Permission.objects.filter(content_type=Tag_CT))

In [47]:
contributors.permissions.all()

<QuerySet [<Permission: organizer | tag | Can add tag>, <Permission: organizer | tag | Can change tag>, <Permission: organizer | tag | Can delete tag>, <Permission: organizer | tag | Can view tag>]>

In [48]:
user.groups.add(contributors)

In [49]:
user = User.objects.get(pk=user.pk)

In [50]:
user.has_perm("organizer.change_tag")

True

In [51]:
user.get_group_permissions()


{'organizer.add_tag',
 'organizer.change_tag',
 'organizer.delete_tag',
 'organizer.view_tag'}

In [52]:
user.get_all_permissions()

{'blog.add_post',
 'blog.change_post',
 'blog.delete_post',
 'blog.view_post',
 'organizer.add_tag',
 'organizer.change_tag',
 'organizer.delete_tag',
 'organizer.view_tag'}

# Notebook Cleanup

In [53]:
user.delete()

(6,
 {'admin.LogEntry': 0,
  'user.User_groups': 1,
  'user.User_user_permissions': 4,
  'user.User': 1})

In [54]:
superuser.delete()

(1,
 {'admin.LogEntry': 0,
  'user.User_groups': 0,
  'user.User_user_permissions': 0,
  'user.User': 1})

In [55]:
contributors.delete()

(5, {'auth.Group_permissions': 4, 'user.User_groups': 0, 'auth.Group': 1})